This notebook walks through how to make graphs in this repo. You can copy this example.ipynb and rename it for your own graphing work. <b> Please do each graph in a seperate notebook!</b>

1. Import the appropriate packages. We'll use plotly.graph_objects for making all of our graphs.

In [1]:
import datetime as dt
import plotly.graph_objects as go
import plotly.io as pio
import pandas as pd
import utils.utils as ut
import utils.graph_templates

from fredapi import Fred

2. We'll get data using the fredapi package. Set the path to the text file with your API key

In [2]:
# Call the graph the exact same thing as its notebook (minus the ipynb suffix) NAME FOR THE FILE, not the title
GRAPH_NAME = "pce_contrib"

API_KEY_PATH = ut.get_repo_root() / "fred_api_key.txt" 

fred = Fred(api_key_file = API_KEY_PATH)

3. Set the fed_2025 template as default

In [3]:
pio.templates.default = 'fed_2025'

# Now is a good time to set the path to the graph output folder!
GRAPH_OUTPUT_PATH = ut.get_repo_root() / "figures"

In [7]:
today = dt.date.today()

contrib_df = pd.read_csv(
    ut.get_repo_root() / "code" / "data" / "chart1-yoy-headline-pce-contributions.csv",
    index_col="date")


headline_srs = fred.get_series(
    series_id="PCEPI",
    observation_start=dt.date(2022, 1, 1),
    observation_end=today,
    frequency='m',
    units='pc1'
).rename("Headline PCE")


fig = go.Figure()

for col in contrib_df.columns:
    fig.add_trace(
        go.Bar(
            x=contrib_df.index,
            y=contrib_df[col],
            name=col
        )
    )


fig.add_trace(
    go.Scatter(
        x=headline_srs.index,
        y=headline_srs,
        mode='lines+markers',
        name='PCE Inflation Rate',
        line=dict(color='black', width=2),
        marker=dict(size=6)
    )
)

# This is graph specific, but here we want the y-axis to be percent signs 
fig.update_yaxes(
    tickformat="0.2f%",
    ticksuffix="%"
)

# Again, graph specific, we have a mutliyear series and want tick marks to be years
fig.update_xaxes(
    type='date',
    tickformat='%b %Y',
)

fig.update_layout(
    barmode='relative',
    title=dict(text='Contributions to PCE Inflation <br><sup>Monthly, Seasonally Adjusted</sup>'),
    xaxis_title='Date',
    yaxis_title='Change from Year Ago (%)',
)

fig.update_layout(
    legend=dict(
        orientation="v",
        x=.7,  # Move legend inside the plot area (left)
        y=1.05,  # Near the top
        xanchor="left",
        yanchor="top",
    )
)

# Add a label at the end of the Headline PCE Inflation series
last_date = headline_srs.index[-1]
last_value = headline_srs.iloc[-1]

fig.add_annotation(
    x=last_date,
    y=last_value + 0.5,
    text=f"{last_value:.1f}%",
    showarrow=False,
    ax=0,
    ay=-200,  # Negative ay moves annotation up (above the point)
    font=dict(size=12, color="black"),
    bgcolor="white",
)


# This should be the same for EVERY GRAPH!
# Save it to the graph_output folder with the name matching the file, as HTML
fig.write_html(GRAPH_OUTPUT_PATH / f"{GRAPH_NAME}.html")

fig.show()


5. Now that all our data is ready, make the graph and have it save itself as a .html file to graph_output whenver the notebooks is rerun. The name of the file should exactly match the notebook name. For instance, this file "example.ipynb" produces the graph "example.html." Nice work, you made a graph! 